In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline,CustomFilter
from quantopian.pipeline.factors import CustomFactor

from quantopian.pipeline.experimental import QTradableStocksUS
from quantopian.pipeline.data.builtin import USEquityPricing

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

class ValueDayBeforeYesterday(CustomFactor):
    window_length = 2
    def compute(self, today, assets, out, values):
        out[:] = values[0]
        
        
def make_pipeline():
    pipe = Pipeline()
    base_universe = QTradableStocksUS()
    pipe.add(USEquityPricing.close.latest, 'close')
    pipe.add(ValueDayBeforeYesterday(inputs = [USEquityPricing.close]), 'prev_close')
    pipe.add(ValueDayBeforeYesterday(inputs = [USEquityPricing.high]), 'prev_high')
    pipe.add(USEquityPricing.open.latest, 'open')
    
    
    pipe.set_screen(base_universe)
    return pipe



In [ ]:
start_date='2013-1-1'
end_date='2018-1-31'
pipeline_results = run_pipeline(make_pipeline(), start_date=start_date, end_date=end_date)

In [ ]:
## ValueDayBeforeYesterdayが正しいかどうか確認
## closeとprev_closeを見比べる
idx = pd.IndexSlice
pipeline_results.loc[idx[:,[symbols("AAPL")]],:].head()


In [ ]:
df = pipeline_results.copy()

In [ ]:
df["Change PrevHigh to Open"] = df["open"]/df["prev_high"]-1
df["Change PrevClose to Open"] = df["open"]/df["prev_close"]-1

thread = 0.05
df["GU from PrevHigh"] = df["Change PrevHigh to Open"] > thread
df["GU from PrevClose"] = df["Change PrevClose to Open"] > thread


In [ ]:
count_high = df["GU from PrevHigh"][df["GU from PrevHigh"]].groupby(level=0).count()
count_close = df["GU from PrevClose"][df["GU from PrevClose"]].groupby(level=0).count()
spy = get_pricing("spy", start_date=count_high.index[0],end_date=count_high.index[-1],
                  fields="price",)

df_count = pd.concat([count_high, count_close, spy], axis=1)

In [ ]:
df_count["spy_return"] = df_count[symbols("spy")].pct_change()

In [ ]:
df_count.plot(kind="scatter", x = "GU from PrevHigh", y = "GU from PrevClose")

In [ ]:
df_count["2013"].drop(["spy_return"], axis=1).plot(secondary_y=symbols("SPY"))

In [ ]:
dates = df[df["GU from PrevHigh"]].index.get_level_values(0).unique()


In [ ]:
df[df["GU from PrevHigh"]].ix[dates[0]]

In [ ]:
minutes_data = []
merged_data = []
dates = df[df["GU from PrevClose"]].index.get_level_values(0).unique()

for date in dates[:100]:
    target = get_pricing(df[df["GU from PrevClose"]].ix[date].index, start_date=date, end_date=date, 
                         frequency='minute', fields="price")
    mn = target.pct_change().cumsum().describe().loc["min"]
    mx = target.pct_change().cumsum().describe().loc["max"]
    gu_high = df[df["GU from PrevClose"]].ix[date]
    merged = pd.concat([mn, mx, gu_high], axis=1)
    minutes_data.append(target)
    merged_data.append(merged)



In [ ]:
plt.scatter( x = pd.concat(merged_data)["min"], y = pd.concat(merged_data)["Change PrevClose to Open"])
plt.scatter( x = pd.concat(merged_data)["max"], y = pd.concat(merged_data)["Change PrevClose to Open"], c='red')